# Dustpy Introduction

you can convert and run this by

    jupyter nbconvert --to python dustpy-intro.ipynb
    python dustpy-intro.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os
import sys

def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')


if is_interactive():
    from IPython import get_ipython
    get_ipython().magic('matplotlib inline')
    
plt.style.use(['seaborn',{'figure.dpi':200}])

After downloading, make sure you have the requirements or install via (in the place where `setup.py` resides)

    pip install -r requirements.txt
    
To install `scikit umfpack` for significant speedup:

    install -c conda-forge scikit-umfpack

Then go to folder and run (where `setup.py` resides)

    pip install -e .
    
For openMP parallelization to work, install it like this:

     pip install -e . --install-option="--build=parallel"

Once this is done, the following should work (ignore the warning that `h5py` might cause):

In [ ]:
import dustpy
from dustpy.sim.utils import bindFunction
from dustpy.sim import constants as c

Create the simulation object

In [ ]:
s = dustpy.sim.Simulation()

Now change whatever you want to change in the settings

In [ ]:
s.pars.verbose=3
s.snapshots=np.array([3.184E-03*c.yr])

We need to tell the code three things:

1. don't evolve the gas viscously
2. use a different initial condition
3. update the gas at every time step with our own function

About part 1:

In [ ]:
s.pars.gasAdvection = False

About part 2 and 3:

In [ ]:
def yourownfunction(r, t, rc, mp1):
    return 200 * (r/rc)**-1 * np.exp(- r/rc)

In [ ]:
# this is how the functions that we bind to the simulation should be defined
def initialGas(sim, r, rc, mp1):
    return yourownfunction(r, 0.0, rc, mp1)

def updateGas(sim, rc, mp1):
    sim.gas.Sigma = yourownfunction(sim.grid.r, sim.t, rc, mp1)

In [ ]:
rc  = 50*c.AU
q   = 1e-3
mp1 = q * c.M_sun

bindFunction(s, 'initialGasSurfaceDensity', initialGas, rc=rc, mp1=mp1)
bindFunction(s, 'gasSystole', updateGas, rc=rc, mp1=mp1)

Now initialize the simulation ...

In [ ]:
if os.path.isdir(s.pars.outputDir):
    yn = ''
    if not is_interactive():
        yn = 'y'
    while yn.lower() not in ['y', 'n']:
        yn = input('output directory exists - delete? ')
    if yn == 'y':
        print('deleting')
        shutil.rmtree(s.pars.outputDir, ignore_errors=True)
    else:
        print('keeping')
s.initialize()

... and finally let it run:

In [ ]:
s.evolve()

In [ ]:
if not is_interactive():
    sys.exit(0)

# Plotting part

Let's do a simple plot - this one uses the attributes of the simulation object, just like your self-defined function should be.

In [ ]:
f, ax = plt.subplots()
ax.loglog(s.grid.r / c.AU, s.gas.Sigma, 'r-', label='gas')
ax.loglog(s.grid.r / c.AU, s.dust.Sigma.sum(1), label='dust')
ax.loglog(s.grid.r / c.AU, yourownfunction(s.grid.r, 0, rc, mp1), 'k--',label='goal')
ax.set_ylim(1e-1,1e4)
ax.legend()
print(f'the initial surface density uses a characteristic radius of r_c = {s.ini.gas.SigmaR0/c.AU} au')

# Plotting results

we need to access the files

In [ ]:
!ls -l {s.pars.outputDir}

we use the HDF5 package `h5py`

In [ ]:
import h5py

Open the file

In [ ]:
f = h5py.File('output/data0000.hdf5')

See what's in it

In [ ]:
list(f)

get some data from the file

In [ ]:
t = f['dt'][()] # the funny brackets are to copy the data into the variable instead of "linking" to the dataset

Some things are actually like sub directories

In [ ]:
list(f['gas'])

One way to access it

In [ ]:
f['gas']['Sigma']

Shorter way:

In [ ]:
sig_g = f['gas/Sigma'][()]

close the file

In [ ]:
f.close()

**The better way to do it**

In [ ]:
with h5py.File('output/data0000.hdf5') as f:
    t = f['dt'][()]
    sig_g = f['gas/Sigma'][()]
    sig_d = f['dust/Sigma'][()]
    r = f['grid/r'][()]
    m = f['grid/m'][()]
    a = f['dust/a'][()]

Plot the results that we just read in

In [ ]:
f, ax = plt.subplots()
ax.loglog(r / c.AU, sig_g)
ax.set_ylim(1e-1,1e4);

Plot the dust surface density

In [ ]:
f, ax = plt.subplots()
cc = ax.contourf(r / c.AU, m, np.log10(sig_d).T, np.arange(-10, 1), extend='both')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title(f'evolution after {t/c.yr:3.3g} yrs')
ax.set_xlabel('radius [AU]')
ax.set_ylabel('particle mass [g]')
plt.colorbar(cc);

## Why binding a function is a bit more complicated

In [ ]:
x = np.array([1,2,3])

In [ ]:
class myclass():
    x = np.linspace(0,10,100)
    t = 10
    def __init__(self, a):
        self.a = a
    def print_a(self):
        print('a is '+str(self.a))
    def give_me_an_alpha(self):
        return self.a * np.ones_like(self.x)

In [ ]:
m = myclass(5)

In [ ]:
m.give_me_an_alpha()

In [ ]:
def my_alpha(s):
    alpha = s.a * np.ones_like(s.x)
    if s.t>50:
        alpha += s.a
    return alpha

In [ ]:
m.t=60

In [ ]:
my_alpha(m)

In [ ]:
m.give_me_an_alpha = my_alpha

In [ ]:
m.give_me_an_alpha()